In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
%matplotlib inline
sns.set(color_codes=True)
from nltk.tokenize import word_tokenize
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [3]:
import os
pwd = os.getcwd()

In [4]:
pwd

'/home/jacksbed/Dropbox/M.Sc. Data Analytics/CAs/CA4/CA2_Submission_folder/M.Sc._Data_Analytics_Semester_2_CA2'

In [5]:
nwd = pwd.removesuffix('/M.Sc._Data_Analytics_Semester_2_CA2')
print(nwd +"/ProjectTweets.csv")

/home/jacksbed/Dropbox/M.Sc. Data Analytics/CAs/CA4/CA2_Submission_folder/ProjectTweets.csv


In [21]:
tweets = pd.read_csv(nwd + "/ProjectTweets.csv", header=None, index_col=0)

In [22]:
tweets.head()

,1,2,3,4,5
0,,,,,
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [23]:
tweets.tail()

,1,2,3,4,5
0,,,,,
1599995,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [24]:
len(tweets)

1600000

In [25]:
#Shows that there are no queries for any of the tweets
tweets[3].unique()

array(['NO_QUERY'], dtype=object)

In [26]:
#Initial mapper was not returning all values because the dates are not always unique
#After this the tweet number was used as the unique identifier. After this the mapper returned 1230523 tweets.
len(tweets[2].unique())

774363

In [27]:
#What I'm calling tweet ID is also not unique to each tweet.
len(tweets[1].unique())

1598315

In [28]:
tweets = tweets.iloc[:,[0,1,3,4]]

In [29]:
tweets.head()

,1,2,4,5
0,,,,
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."


In [30]:
tweets = tweets.reset_index(drop=True)

In [31]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   1       1600000 non-null  int64 
 1   2       1600000 non-null  object
 2   4       1600000 non-null  object
 3   5       1600000 non-null  object
dtypes: int64(1), object(3)
memory usage: 48.8+ MB


In [32]:
tweets.head()

,1,2,4,5
0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...
2,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...
3,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire
4,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all...."


In [33]:
tweets = tweets.rename(columns = {1:'Tweet number', 2:'Date', 4:'Username', 5:'tweet'})

In [34]:
tweets.tail()

,Tweet number,Date,Username,tweet
1599995,2193601966,Tue Jun 16 08:40:49 PDT 2009,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,2193601969,Tue Jun 16 08:40:49 PDT 2009,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,2193601991,Tue Jun 16 08:40:49 PDT 2009,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,2193602064,Tue Jun 16 08:40:49 PDT 2009,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,2193602129,Tue Jun 16 08:40:50 PDT 2009,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [39]:
ids = tweets["Tweet number"]
tweets[ids.isin(ids[ids.duplicated()])].sort_values("Tweet number")

,Tweet number,Date,Username,tweet
213,1467863684,Mon Apr 06 22:33:35 PDT 2009,DjGundam,Awwh babs... you look so sad underneith that s...
800261,1467863684,Mon Apr 06 22:33:35 PDT 2009,DjGundam,Awwh babs... you look so sad underneith that s...
275,1467880442,Mon Apr 06 22:38:04 PDT 2009,iCalvin,Haven't tweeted nearly all day Posted my webs...
800300,1467880442,Mon Apr 06 22:38:04 PDT 2009,iCalvin,Haven't tweeted nearly all day Posted my webs...
989,1468053611,Mon Apr 06 23:28:09 PDT 2009,mariejamora,@hellobebe I also send some updates in plurk b...
...,...,...,...,...
524290,2193428118,Tue Jun 16 08:26:44 PDT 2009,bubbles00,"finally finished typing!!!! Woohoooo , still ..."
1599440,2193451289,Tue Jun 16 08:28:37 PDT 2009,faiznurdavid,"@fanafatin see, @misschimichanga tweet u to jo..."
524345,2193451289,Tue Jun 16 08:28:37 PDT 2009,faiznurdavid,"@fanafatin see, @misschimichanga tweet u to jo..."
524688,2193576442,Tue Jun 16 08:38:45 PDT 2009,emma_b_xox,Had an injection today. Not fun the rrst of t...


## Clean the tweet data

Remove punctuation

Vader takes capitalisation into account when determining sentiment, so case will not be changed

This was later done with mapreduce (much more rapidly)

In [17]:
#tweet = re.sub(r'[^\w\s]', '', tweets.tweet[1]).lower()
tweet = re.sub(r'[^\w\s]', '', tweets.tweet[1])

In [18]:
tweet

'is upset that he cant update his Facebook by texting it and might cry as a result  School today also Blah'